In [3]:
import pygame
import numpy as np
import sympy as sp
import random
import pandas as pd



pygame.init()

BOARD_SIZE_X = 5
BOARD_SIZE_Y = 10
TILE_SIZE_X = 200
TILE_SIZE_Y = 60
TOTAL_COUNT = 0
GOAL_COUNT = 0

WIDTH, HEIGHT = BOARD_SIZE_X * TILE_SIZE_X, BOARD_SIZE_Y * TILE_SIZE_Y
screen = pygame.display.set_mode((WIDTH, HEIGHT))
pygame.display.set_caption("DeepBalls")

# Colors
GREEN_ONE = (8, 102, 38)
GREEN_TWO = (0, 255, 82)
RED = (255, 0, 0)
BLUE = (0, 0, 255)
BLACK = (0, 0, 0)
WHITE = (255, 255, 255)

board = np.zeros((BOARD_SIZE_X, BOARD_SIZE_Y), dtype=int)

if_goal = False

running = True

# Arrow configurations
arrow_offsets = {
    BLUE: [(1.7, 0), (1, 1), (1, -1)],
    RED: [(-1.7, 0), (-1, 1), (-1, -1)]
}

new_directions = {
    BLUE: [(1, 0), (1, 1), (1, -1)],
    RED: [(-1, 0), (-1, 1), (-1, -1)]
}

arrows = []


class Player:
    def __init__(self, x, y, color):
        self.position = (x * TILE_SIZE_X, y * TILE_SIZE_Y)
        self.color = color


# 定义黑色球员和红色球员的位置
black_players = [
    Player(3, 5, BLUE),
    Player(1, 8, BLUE),
    Player(1, 2, BLUE)
]

red_players = [
    Player(2, 5, RED),
    Player(4, 2, RED),
    Player(4, 8, RED)
]

# 合并球员列表
players = black_players + red_players

columns = ['Ball Position X', 'Ball Position Y', 'Ball Direction X', 'Ball Direction Y']
for i in range(len(players)):
    columns.extend([f'Player {i} Position X', f'Player {i} Position Y'])
columns.append('Steps to Goal')
    
df_temp = pd.DataFrame(columns=columns)
df = pd.DataFrame(columns=columns)

class Ball:
    def __init__(self, x, y, direction):
        self.position = (x * TILE_SIZE_X, y * TILE_SIZE_Y)
        self.direction = direction


# 定义球的位置和方向
ball = Ball(3, 5, (1, 0))  # 球的位置在 (3, 5)，方向为向右


# 绘制棋盘
def draw_board():
    for row in range(BOARD_SIZE_Y):
        for col in range(BOARD_SIZE_X):
            color = BLACK
            pygame.draw.rect(screen, color,
                             pygame.Rect(col * TILE_SIZE_X, row * TILE_SIZE_Y, TILE_SIZE_X - 2, TILE_SIZE_Y - 2))


# 绘制球员
def draw_players():
    for player in players:
        col, row = player.position
        pygame.draw.circle(screen, player.color,
                           (col, row),
                           TILE_SIZE_Y // 4)


# 绘制球和箭头
def draw_ball_and_arrow():
    col, row = ball.position
    pygame.draw.circle(screen, WHITE,
                       (int(col), int(row)),
                       TILE_SIZE_Y // 4)

    # 绘制箭头
    arrow_start = (int(col), int(row))
    arrow_end = (int(arrow_start[0] + ball.direction[0] * 20),
                 int(arrow_start[1] + ball.direction[1] * 20))
    pygame.draw.line(screen, WHITE, arrow_start, arrow_end, 5)
    pygame.draw.polygon(screen, WHITE, [
        (int(arrow_end[0] + ball.direction[0] * 10), int(arrow_end[1] + ball.direction[1] * 10)),
        (int(arrow_end[0] - ball.direction[1] * 10), int(arrow_end[1] + ball.direction[0] * 10)),
        (int(arrow_end[0] + ball.direction[1] * 10), int(arrow_end[1] - ball.direction[0] * 10))
    ])

    for arrow in arrows:
        draw_arrow(arrow['start'], arrow['end'], arrow['color'])


def draw_arrow(start, end, color):
    pygame.draw.polygon(screen, color, [
        (int(end[0] + (end[0] - start[0]) * 0.2), int(end[1] + (end[1] - start[1]) * 0.2)),
        (int(end[0] - (end[1] - start[1]) * 0.2), int(end[1] + (end[0] - start[0]) * 0.2)),
        (int(end[0] + (end[1] - start[1]) * 0.2), int(end[1] - (end[0] - start[0]) * 0.2))
    ])


def show_arrows(player):
    global arrows
    x, y = player.position
    arrows = []
    for offset in arrow_offsets[player.color]:
        end_x = x + offset[0] * 20
        end_y = y + offset[1] * 20
        arrows.append({
            'start': (x, y),
            'end': (end_x, end_y),
            'color': player.color,
            'direction': offset
        })


# 处理球的运动逻辑
def move_ball():
    global arrows, GOAL_COUNT, if_goal
    x, y = sp.symbols('x y')
    col, row = ball.position
    direction = ball.direction

    # 定义球的运动方程
    line_eq = sp.Eq((y - row) * direction[0], (x - col) * direction[1])

    # print(ball.position, ball.direction)

    # 寻找最近的交点
    intersection_points = []
    for i in range(BOARD_SIZE_X + 1):
        if direction[0] != 0:
            x_val = i * TILE_SIZE_X
            if ball.direction[0] * (x_val - col) > 0:
                y_val = sp.solve(line_eq.subs(x, x_val), y)[0]
                formatted_tuple = tuple(format(x, '.20f') for x in ball.position)
                if 0 <= y_val <= HEIGHT:
                    intersection_points.append((x_val, y_val))

    for j in {0, 10}:
        if direction[1] != 0:
            y_val = j * TILE_SIZE_Y
            if ball.direction[1] * (y_val - row) > 0:
                x_val = sp.solve(line_eq.subs(y, y_val), x)[0]
                if 0 <= x_val <= WIDTH:
                    intersection_points.append((x_val, y_val))

    # 找到最近的交点
    next_position = min(intersection_points, key=lambda p: ((p[0] - col) ** 2 + (p[1] - row) ** 2) ** 0.5)

    if_goal = False

    # 检查碰撞边框
    if int(next_position[0]) == 0 or int(next_position[0]) == WIDTH:
        direction = (-direction[0], direction[1])
        if 240 <= next_position[1] <= 360:
            GOAL_COUNT += 1
            if_goal = True
    if int(next_position[1]) == 0 or int(next_position[1]) == HEIGHT:
        direction = (direction[0], -direction[1])

    ball.position = next_position
    ball.direction = direction

    for p in players:  # to do 边界时随机可以改成二选一, 应该拆出去
        new_y = p.position[1] + random.choice([-TILE_SIZE_Y, 0, TILE_SIZE_Y])
        if 0 <= new_y <= HEIGHT:
            p.position = (p.position[0], new_y)


def move_players():
    for p in players:
        if ((p.position[0] - ball.position[0]) ** 2 + (p.position[1] - ball.position[1]) ** 2) ** 0.5 < 40:
            # print("shoot!!!!!")
            show_arrows(p)
            ball.direction = random.choice(new_directions[p.color])
            break
        
def update_dataframe():
    global df, df_temp
    row = [
        ball.position[0], ball.position[1], ball.direction[0], ball.direction[1]
    ]
    for player in players:
        row.extend([player.position[0], player.position[1]])
    row.append(0)
    df_temp = pd.concat([df_temp, pd.DataFrame([row], columns=df.columns)], axis=0)
    if if_goal:
        if int(ball.position[0]) != 0:
            df = pd.concat([df, df_temp], axis=0)
        df_temp = pd.DataFrame(columns=columns)
        
    else:
        df_temp["Steps to Goal"] += 1
    df.to_csv('output.csv', index=False)



# 处理鼠标点击事件
def handle_click(pos):
    global ball
    x, y = pos

    arrows.clear()
    move_ball()
    move_players()
    update_dataframe()

while running:
    for event in pygame.event.get():
        if event.type == pygame.QUIT:
            running = False
        elif event.type == pygame.MOUSEBUTTONDOWN:
            TOTAL_COUNT += 1
            handle_click(pygame.mouse.get_pos())
            print(GOAL_COUNT, TOTAL_COUNT)

    screen.fill(GREEN_ONE)
    draw_board()
    draw_players()
    draw_ball_and_arrow()
    pygame.display.flip()

pygame.quit()


0 1
1 2
1 3
1 4
1 5
1 6
2 7
2 8
2 9
2 10
2 11
3 12
3 13
3 14
3 15
3 16
3 17
3 18
3 19
3 20
3 21
3 22
3 23
3 24
3 25
3 26
3 27
3 28
3 29
3 30
3 31
3 32
3 33
3 34
4 35
4 36
4 37
4 38
4 39
4 40
4 41
4 42
4 43
4 44
4 45
4 46
4 47
4 48
4 49
4 50
4 51
4 52
4 53
4 54
5 55
5 56
5 57
5 58
5 59
5 60
5 61
5 62
5 63
5 64
5 65
5 66
5 67
5 68
5 69
5 70
5 71
5 72
5 73
5 74
6 75
6 76
6 77
6 78
6 79
6 80
6 81
6 82
6 83
6 84
6 85
6 86
6 87
6 88
6 89
6 90
6 91
6 92
6 93
6 94
7 95
7 96
7 97
7 98
7 99
7 100
8 101
8 102
8 103
8 104
8 105
8 106
8 107
8 108
8 109
8 110
8 111
8 112
8 113
8 114
8 115
8 116
8 117
8 118
8 119
8 120
9 121
9 122
9 123
9 124
9 125
9 126
9 127
9 128
9 129
9 130
9 131
9 132
9 133
9 134
9 135
9 136
9 137
9 138
9 139
9 140
9 141
9 142
9 143
9 144
9 145
9 146
9 147
9 148
9 149
9 150
9 151
9 152
9 153
10 154
10 155
10 156
10 157
10 158
10 159
10 160
10 161
10 162
10 163
10 164
10 165
10 166
10 167
10 168
10 169
10 170
10 171
10 172
10 173
11 174
11 175
11 176
11 177
11 178
11 179
11 180
1